# RRN-LSTM with Tensorflow

Following the tutorial [A noob’s guide to implementing RNN-LSTM using Tensorflow](http://monik.in/a-noobs-guide-to-implementing-rnn-lstm-using-tensorflow).

See [Andrej Karpathy's blog post](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) for more information on RRNs.

# Task
Given a binary string, containing `0`s and `1`s, of length 20, we need to determine the count of `1`s the string.

# Simple Solution
This is easy to solve without using a RRN, as shown in the code below. However, this is a simple problem with which to experiment using RRNs.

In [1]:
from functools import reduce

def count_num_1s(s: str) -> int:    
    return reduce((lambda acc, x: acc + x if x == 1 else acc), s)

# Should print out '9'.
print(str(count_num_1s([1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1])))

9


# RRN Solution

## All Possible Data

- Each input is a binary string of length 10.
- An input will be represented as a python list of `0`s and `1`s.
- There are $2^{10}$ possible combinations of `0`s and `1`s. 
- The network will be trained on a subset all possible combinations.
- The remainder of the data will be used for testing.

In [2]:
import numpy as np
from typing import List

def make_all_binary_strings(length: int = 10) -> List[List[int]]:
    """
    returns: a list of all possible binary strings, where a binary string is represened as a list of ints.
    """
    format_string = '{0:0' + str(length) + 'b}'
    all_binary_strings = [format_string.format(i) for i in range(2**length)]
    return [list(map(int,i)) for i in all_binary_strings]

example_all_arrs = make_all_binary_strings(length=2)
print(example_all_arrs)

[[0, 0], [0, 1], [1, 0], [1, 1]]


- Tensorflow requires input as a tensor, i.e. a Tensorflow variable.
- The tensor is 3D with the dimensions [batch_size, sequence_length, input_dimension], where
    - `batch_size` is something we’ll determine later 
    - `sequence_length` is fixed at 10 
    - `input_dimension` is 1 i.e each individual bit of the string. Therefore, each bit will actually be represented as a list containing just that bit. 

In [3]:
def make_all(binary_strings: List[List[int]]) -> List[np.array]:
    # Turn every every element into an array containing that one element.
    # This is required as the input_dimension is 1.
    xs = list(map(lambda bits: list(map(lambda bit: [bit], bits)), binary_strings))
    # Make each array of bits a numpy array.
    return list(map(lambda bits: np.array(bits), xs))

example_all = make_all(example_all_arrs)
print(example_all)

[array([[0],
       [0]]), array([[0],
       [1]]), array([[1],
       [0]]), array([[1],
       [1]])]


### Labelling Training Data
- For every sequence, the result can be anything between 0 and 20. 
- Therefore there are have 21 categories in which a sequence can be classified into.
- Each sequence belongs to the class number which is the same as the count of ones in the sequence.
- The output is represented using 1-hot encoding.
- i.e. a vector containing all zeros except of one position which containings a one, indicating the class the sequence belongs to.

For example, the sequence `0011` would have the top output vector below, indicating there are two `1`s in the sequence.

`[0 0 1 0 0]`

`[0 1 2 3 4]` (the number of `1`s)

The length of the output vector is the length of the binary sequence + 1.

In [4]:
def label(data: List[np.array]) -> List[int]:
    def f(bits: np.array):
        # We made each element a single array in `make_training`.
        # By flattening we can more easily work with the data.
        flattened = bits.flatten()
        # The index in the one-hot vector that should be set to 1.
        i = count_num_1s(flattened)
        # Create the vector and set the class.
        vec = [0] * (len(bits) + 1)
        vec[i] = 1
        return vec
    
    return list(map(f, data))

example_labels = label(example_all)
print(list(zip(example_all_arrs, example_labels)))

[([0, 0], [1, 0, 0]), ([0, 1], [0, 1, 0]), ([1, 0], [0, 1, 0]), ([1, 1], [0, 0, 1])]


# Training and Testing Data
- The full data set is split into training and testing data.
- These sets don't overlap.

In [5]:
from typing import Tuple
from random import shuffle

def split(data: List[np.array], num_training: int) -> Tuple[List[np.array], List[np.array]]:
    """
    data: a list of the data to be split into training and testing.
    proportion_testing: the proportion in which the data will be split in two. 
    returns: shuffled data split into training and testing data.
    """    
    # shuffle works in place, therefore copy the data.
    shuffled_data = list(data)
    shuffle(shuffled_data)
    
    training = shuffled_data[:num_training]
    testing  = shuffled_data[num_training:]
    
    return (training, testing)

example_training, example_testing = split(example_all, num_training=3)

print("Training: " + str(example_training))
print("\nTesting: " + str(example_testing))

Training: [array([[1],
       [1]]), array([[0],
       [1]]), array([[1],
       [0]])]

Testing: [array([[0],
       [0]])]


# Tensorflow Model

- Define two variables for data.
- The dimensions for the data are $[\text{Batch Size}, \text{Sequence Length}, \text{Input Dimension}]$. 
- `training_data` has sequence length of 10 and input dimension of 1, as defined earlier.
- `training_labels` has sequence length of 11 for the one-hot encoding representing all the possible classes (0, 1, 2, etc)
- The batch size is determined at runtime, and hence is `None`.
- Placeholders tell Tensorflow that the data will be supplied later.
- Build the model first, then run it right at the end.

In [6]:
import tensorflow as tf

sequence_length = 16
data = tf.placeholder(tf.float32, [None, sequence_length, 1])
target = tf.placeholder(tf.float32, [None, sequence_length + 1])

## LSTM Cell
- For each LSTM cell that we initialise, we need to supply a value for the hidden dimension
- i.e. the number of units in the LSTM cell. 
- If the value os too high a value may lead to overfitting
- If the value is too low value may yield poor results

In [7]:
num_hidden = 24
cell = tf.nn.rnn_cell.LSTMCell(num_hidden,state_is_tuple=True)

## Graph
- Unroll the network, pass data to it, and store the value in `val`.
- `state` is discarded (never used again).
- This doesn't run the NN.
- These represent functions that are stored in variables.
- The functions are run when a session is started.

In [8]:
val, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)

## Class Labels
- Transpose to switch batch size and sequence size and store the result in `val`.
- This is done so we can get the output at the sequence's last input.
- i.e. in a string of 10 bits we're only interested in the output (class) we got at the 20th character.
- Class labels are generated as we're going along because we're using a RNN (?).

In [9]:
val = tf.transpose(val, [1, 0, 2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)

## Predictions

In order to apply the final transformation to the outputs of the LSTM and map it to the output classes (i.e. number of `1`s), we need the following variables:

In [10]:
weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))

- `tf.matmul(last, weight) + bias` gives matrix with a variety of values for each class.
- i.e. the class for each sequence (?)
- We are interested in the probability score for each class.
- The value represents the probablilty the sequence belongs to a particular class.
- i.e the chance that the sequence belongs to a particular class
- This is done using the softmax activation function.
- This function takes in a vector of values and returns a probability distribution for each index depending upon its value.

In [11]:
prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)

## Loss (Correctness)

- Calculate the loss.
- i.e. the degree of incorrectness.
- The cost function determines how poorly or how well our predictions stack against the actual results (labels).

In [12]:
cross_entropy = -tf.reduce_sum(target * tf.log(tf.clip_by_value(prediction,1e-10,1.0)))

## Optimisation
- Use an optimisation function to optimise the network.
- i.e. set weights on edges (?)

In [13]:
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

/Users/albiebakersmith/anaconda3/envs/rrn_test/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


# Tensorflow Testing 
- The below error is a count of how many sequences in the test dataset were classified incorrectly. 
- This gives us an idea of the correctness of the model on the test dataset.

In [14]:
mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

## Generate data
Runs the functions created before to generate training and testing data.

In [15]:
all_arrs = make_all_binary_strings(length=sequence_length)
all_data = make_all(all_arrs)

num_training = 10000#int(len(all_data) * 0.1)
print("Num training = " + str(num_training))
print("Num testing  = " + str(len(all_data) - num_training))

train_input, test_input = split(all_data, num_training=15)

train_output = label(train_input)
test_output = label(test_input)

Num training = 10000
Num testing  = 55536


Finally, we can execute the model.

In [16]:
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

In [17]:
batch_size = 1000
no_of_batches = int(len(train_input)/batch_size)
epoch = 5000
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        sess.run(minimize,{data: inp, target: out})
#     print("Epoch - ", str(i))

In [18]:
incorrect = sess.run(error,{data: test_input, target: test_output})
print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))

Epoch 5000 error 98.6%


In [20]:
print(sess.run(prediction,{data: [[[1],[0],[0],[1],[1],[0],[1],[1],[1],[0],[1],[0],[0],[1],[1],[0]]]}))

[[ 0.02869153  0.08481354  0.03939343  0.04407435  0.03804926  0.06192828
   0.0629071   0.07557759  0.04023664  0.06145936  0.03298585  0.07052813
   0.03663613  0.09131815  0.04738227  0.07781862  0.10619969]]


In [ ]:
sess.close()